# Mozilla TTS Synthesis with GL
## Use your model 

### Set the paths for your model


In [4]:
# model paths
TTS_MODEL = "data/tts_model.pth.tar"
TTS_CONFIG = "data/config.json"
VOCODER_MODEL = "data/vocoder_model.pth.tar"
VOCODER_CONFIG = "data/config_vocoder.json"

In [25]:
TTS_MODEL = "/home/julian/workspace/train/Multilingual_multispeaker-April-16-2021_12+34AM-904c6b1/best_model.pth.tar"
TTS_CONFIG = "/home/julian/workspace/train/Multilingual_multispeaker-April-16-2021_12+34AM-904c6b1/config.json"
TTS_LANGUAGES = "/home/julian/workspace/train/Multilingual_multispeaker-April-16-2021_12+34AM-904c6b1/languages.json"
TTS_SPEAKERS = "/home/julian/workspace/train/Multilingual_multispeaker-April-16-2021_12+34AM-904c6b1/speakers.json"

In [12]:
TTS_MODEL = "/home/julian/workspace/train/Multilingual_multispeaker_GST-April-17-2021_01+27PM-904c6b1/checkpoint_30000.pth.tar"
TTS_CONFIG = "/home/julian/workspace/train/Multilingual_multispeaker_GST-April-17-2021_01+27PM-904c6b1/config.json"
TTS_LANGUAGES = "/home/julian/workspace/train/Multilingual_multispeaker_GST-April-17-2021_01+27PM-904c6b1/languages.json"
TTS_SPEAKERS = "/home/julian/workspace/train/Multilingual_multispeaker_GST-April-17-2021_01+27PM-904c6b1/speakers.json"

### Define TTS function

In [26]:
def tts(model, text, CONFIG, use_cuda, ap, speaker_id=None, language_id=None, language_mapping=None, figures=True):
    t_1 = time.time()
    waveform, alignment, mel_spec, mel_postnet_spec, stop_tokens, inputs = synthesis(model, text, CONFIG, use_cuda, ap, speaker_id=speaker_id, language_id=language_id, language_mapping=language_mapping, style_wav=None, truncated=False, enable_eos_bos_chars=CONFIG.enable_eos_bos_chars, use_griffin_lim=True)
    rtf = (time.time() - t_1) / (len(waveform) / ap.sample_rate)
    tps = (time.time() - t_1) / len(waveform)
    print(waveform.shape)
    print(" > Run-time: {}".format(time.time() - t_1))
    print(" > Real-time factor: {}".format(rtf))
    print(" > Time per step: {}".format(tps))
    IPython.display.display(IPython.display.Audio(waveform, rate=CONFIG.audio['sample_rate']))  
    return alignment, mel_postnet_spec, stop_tokens, waveform

### Load Models

In [27]:
import os
import torch
import time
import IPython

from TTS.tts.utils.generic_utils import setup_model
from TTS.utils.io import load_config
from TTS.tts.utils.speakers import load_speaker_mapping, load_language_mapping
from TTS.tts.utils.text.symbols import symbols, phonemes
from TTS.utils.audio import AudioProcessor
from TTS.tts.utils.synthesis import synthesis

In [28]:
# runtime settings
use_cuda = True

In [29]:
# load config
TTS_CONFIG = load_config(TTS_CONFIG)

In [30]:
# load the audio processor
TTS_CONFIG.audio['stats_path'] = None
ap = AudioProcessor(**TTS_CONFIG.audio)         

self.log_func = np.log10
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:27
 | > do_sound_norm:False
 | > stats_path:None
 | > log_func:<ufunc 'log10'>
 | > exp_func:<function AudioProcessor.__init__.<locals>.<lambda> at 0x7f98dda27af0>
 | > hop_length:256
 | > win_length:1024


In [31]:
# LOAD TTS MODEL

# Load speakers and languages
speaker_mapping = load_speaker_mapping(TTS_SPEAKERS)
language_mapping = load_language_mapping(TTS_LANGUAGES)

# load the model
num_chars = len(phonemes) if TTS_CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, len(speaker_mapping), len(language_mapping), TTS_CONFIG)

# load model state
cp =  torch.load(TTS_MODEL, map_location=torch.device('cpu'))

# load the model
model.load_state_dict(cp['model'])
if use_cuda:
    model.cuda()
model.eval()

# set model stepsize
if 'r' in cp:
    model.decoder.set_r(cp['r'])

 > Using model: Tacotron2


In [32]:
speaker_mapping

{'bernard': 0,
 'elizabeth_klett': 1,
 'elliot_miller': 2,
 'eva_k': 3,
 'ezwa': 4,
 'gilles_g_le_blanc': 5,
 'hajdurova': 6,
 'judy_bieber': 7,
 'karen_savage': 8,
 'karlsson': 9,
 'lisa_caputo': 10,
 'mary_ann': 11,
 'minaev': 12,
 'nadine_eckert_boulet': 13,
 'nikolaev': 14,
 'nina_brown': 15,
 'piotr_nater': 16,
 'ramona_deininger': 17,
 'rebecca_braunert_plunkett': 18,
 'riccardo_fasol': 19,
 'tux': 20,
 'victor_villarraza': 21,
 'zeckou': 22}

In [33]:
language_mapping

{'de': 0,
 'en-gb': 1,
 'en-us': 2,
 'es': 3,
 'fr-fr': 4,
 'it': 5,
 'pl': 6,
 'ru': 7}

## Run Inference

In [70]:
sentence =  "Bill a pris l'habitude de se demander. Cette pensée est-elle vraie ? et s'il n'en était pas absolument certain, il la laissait tomber."
align, spec, stop_tokens, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, speaker_id=4, language_id=4, language_mapping=language_mapping, figures=True)

[WARNING] 1 utterances containing language switches on lines 1
[WARNING] extra phones may appear in the "fr-fr" phoneset
[WARNING] language switch flags have been removed (applying "remove-flags" policy)


(84736,)
 > Run-time: 1.5293927192687988
 > Real-time factor: 0.28877532013953633
 > Time per step: 1.804846876337449e-05


In [71]:
sentence =  "Bill machte es sich zur Gewohnheit, sich selbst zu fragen. Ist dieser Gedanke wahr? und wenn er sich nicht absolut sicher wäre, würde er ihn fallen lassen."
align, spec, stop_tokens, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, speaker_id=18, language_id=0, language_mapping=language_mapping, figures=True)

(113408,)
 > Run-time: 1.556081771850586
 > Real-time factor: 0.21952572310213042
 > Time per step: 1.3720387126199279e-05


In [72]:
sentence =  "Bill got into the habit of asking himself. Is that thought true? and if he wasn't absolutely certain it was, he would let it go."
align, spec, stop_tokens, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, speaker_id=1, language_id=1, language_mapping=language_mapping, figures=True)

(94976,)
 > Run-time: 1.350890874862671
 > Real-time factor: 0.22756137616550826
 > Time per step: 1.4222621154592365e-05


In [36]:
sentence =  "Then the North Wind blew as hard as he could, but the more he blew the more closely did the traveler fold his cloak around him;"
align, spec, stop_tokens, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, speaker_id=11, language_id=2, language_mapping=language_mapping, figures=True)

(114432,)
 > Run-time: 1.640629529953003
 > Real-time factor: 0.2293830083253933
 > Time per step: 1.4336454688302622e-05


In [37]:
sentence =  "Bóreas empezó de primero, soplando con violencia; y apretó el hombre contra sí sus ropas"
align, spec, stop_tokens, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, speaker_id=20, language_id=3, language_mapping=language_mapping, figures=True)

(77568,)
 > Run-time: 1.4376041889190674
 > Real-time factor: 0.29651722302137823
 > Time per step: 1.8532341807195457e-05


In [75]:
sentence =  "Bill ha preso l'abitudine di chiedersi. È vero quel pensiero? e se non era assolutamente certo che lo fosse, lasciava perdere."
align, spec, stop_tokens, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, speaker_id=19, language_id=5, language_mapping=language_mapping, figures=True)

(91904,)
 > Run-time: 1.3231024742126465
 > Real-time factor: 0.23033061067373972
 > Time per step: 1.439569689188163e-05


In [76]:
sentence =  "Билл вошёл в привычку спрашивать себя. Это правда? И если бы он не был абсолютно уверен в этом, он бы отпустил это."
align, spec, stop_tokens, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, speaker_id=16, language_id=7, language_mapping=language_mapping, figures=True)

(98048,)
 > Run-time: 1.3689019680023193
 > Real-time factor: 0.2233700403656723
 > Time per step: 1.396065913432572e-05


In [77]:
sentence =  "Bill nabrał nawyku zadawania sobie pytania. Czy ta myśl jest prawdziwa? I jeśli nie był absolutnie pewien, że tak, to odpuszczał."
align, spec, stop_tokens, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, speaker_id=15, language_id=6, language_mapping=language_mapping, figures=True)

(102144,)
 > Run-time: 1.4859898090362549
 > Real-time factor: 0.23275445726581084
 > Time per step: 1.454717925467288e-05
